In [4]:
import healpy as hp
import numpy as np
import shapely.vectorized
import csv
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import matplotlib.colors as mcolors
import math
from shapely.geometry import Point, Polygon

In [5]:
def getBoundaries(order, i):
    x_coor = []
    y_coor = []
    z_coor = []
    # for i in range(hp.order2npix(order)):
    test = hp.boundaries(hp.order2nside(order), i, step=1)
    x_coor.extend(test[0])
    y_coor.extend(test[1])
    z_coor.extend(test[2])
    
    return x_coor, y_coor, z_coor

In [6]:
def getArcs(points):
    
    p1, p2, p3, p4= np.transpose(points)
    arc12 = great_circle_arc(p1, p2)
    arc23 = great_circle_arc(p2, p3)
    arc34 = great_circle_arc(p3, p4)
    arc41 = great_circle_arc(p4, p1)
    
    return arc12, arc23, arc34, arc41

In [7]:
def getPoints(arc12, arc23, arc34, arc41, num_points = 50):
    new_points = []
    all_arc_lengths = []
    
    #This function returns arcs both from top to bottom as well as side to side
    for i in range(num_points):
        arc = great_circle_arc(arc12[i], arc34[num_points - 1 - i])
        new_points.extend(arc)
        arc_lengths=[]
        # for j in range(49):
        #     arc_lengths.append(spherical_angle_diff(arc[j], arc[j+1]))
        # all_arc_lengths.append(arc_lengths)
    for i in range(num_points):
        arc = great_circle_arc(arc23[i], arc41[num_points -1 - i])
        new_points.extend(arc)
        arc_lengths=[]
        # for j in range(49):
        #     arc_lengths.append(spherical_angle_diff(arc[j], arc[j+1]))
        # all_arc_lengths.append(arc_lengths)


    

                
                    
    return np.array(new_points)
#, np.array(all_arc_lengths)

In [8]:
def great_circle_arc(p1, p2, num_points=50):
    """Generates points along a great circle arc between two points."""
    p1_norm = p1 / np.linalg.norm(p1)
    p2_norm = p2 / np.linalg.norm(p2)
    points = []
    for i in range(num_points):
        t = i / (num_points - 1)
        # Linear interpolation and normalization
        pt = (1 - t) * p1_norm + t * p2_norm
        pt_norm = pt / np.linalg.norm(pt)
        points.append(pt_norm)
    return np.array(points)

In [9]:
def plot_points_on_sphere(points, new_points, order, sphere_opacity=0.1):
    """
    Plot points on the surface of a sphere with interactive rotation.
    Parameters:
    - df (pandas DataFrame): DataFrame containing 'pixel', 'longitude', 'latitude'.
    - nside (int): HEALPix nside parameter.
    - sphere_opacity (float): Opacity of the sphere surface (0 to 1).
    """
    #num_pixels = hp.nside2npix(nside)
    #colors = assign_colors(df['pixel'].values, num_pixels)
    # Convert lon, lat to Cartesian coordinates
    x = points[0] 
    y = points[1]
    z = points[2]
    # Create scatter3d plot
    scatter = go.Scatter3d(
        x=x,
        y=y,
        z=z,
        mode='markers',
        marker=dict(
            size=5,
            color="red",
            colorscale='Viridis',
            opacity=0.8,
            colorbar=dict(title='Pixel Index')
        ),
        name='HEALPix Points'
    )
    x1 = new_points[:,0]
    y1 = new_points[:,1]
    z1 = new_points[:,2]
    # Create scatter3d plot
    scatter1 = go.Scatter3d(
        x=x1,
        y=y1,
        z=z1,
        mode='markers',
        marker=dict(
            size=2,
            color="yellow",
            colorscale='Viridis',
            opacity=0.8,
            colorbar=dict(title='Pixel Index')
        ),
        name='HEALPix Points'
    )
    # Create a sphere surface for reference
    u = np.linspace(0, 2 * np.pi, 100)
    v = np.linspace(0, np.pi, 100)
    xs = np.outer(np.cos(u), np.sin(v))
    ys = np.outer(np.sin(u), np.sin(v))
    zs = np.outer(np.ones(np.size(u)), np.cos(v))
    sphere = go.Surface(
        x=xs,
        y=ys,
        z=zs,
        colorscale='Gray',
        opacity=sphere_opacity,
        showscale=False,
        name='Sphere'
    )
    # Combine scatter and sphere in the figure
    fig = go.Figure(data=[sphere, scatter, scatter1])
    # Update layout for better visualization
    fig.update_layout(
        title=f'HEALPix Points on Sphere (order={order})',
        scene=dict(
            xaxis=dict(showbackground=False, visible=False),
            yaxis=dict(showbackground=False, visible=False),
            zaxis=dict(showbackground=False, visible=False),
            aspectmode='data'
        ),
        width=800,
        height=800
    )
    # Display the plot
    fig.show()

In [10]:
def main(order):
    combinedPoints = []
    combinedNewPoints = []
    for i in range(12):
        points = getBoundaries(order, i)
        combinedPoints.append(points)
        arc12, arc23, arc34, arc41 = getArcs(points)

        new_points = getPoints(arc12, arc23, arc34, arc41, 50)
        combinedNewPoints.append(new_points)
    print(np.shape)
    combinedPoints = np.concatenate(combinedPoints, axis=0)
    combinedNewPoints = np.concatenate(combinedNewPoints, axis = 0)
    plot_points_on_sphere(combinedPoints, combinedNewPoints, order)
        

In [11]:
main(0)

<function shape at 0x702604177380>


In [12]:
def single_main(order, i):


    points = getBoundaries(order, i)

    arc12, arc23, arc34, arc41 = getArcs(points)

    new_points = getPoints(arc12, arc23, arc34, arc41, 50)


    plot_points_on_sphere(points, new_points, order)


In [13]:
single_main(0,0)